# แก้ไขรายชื่อผู้ทรงคุณวุฒิที่ มธ. เป็นผู้จัดเก็บ
เปิดไฟล์ Excel ข้อมูลของ มธ. และตั้งชื่อไฟล์สำหรับเก็บข้อมูลที่มีปัญหา

In [1]:
import os
import pandas as pd
from fixblank import rmblankname
from fixwords import fixranks, fixphd, fixuni, fixstatus
from fixnames import fix_name, get_title
from fixtitle import fix_title
pd.set_option('display.max_rows', 10)
fn = 'รายชื่ออาจารย์มหาวิทยาลัย_เว็บไซต์.xlsx'
output_file = os.path.splitext(os.path.basename(fn))[0] + '_broken.xlsx'
xl = pd.ExcelFile(fn)

กำหนดคอลัมน์ที่จำเป็น และชื่อคอลัมน์ใหม่ที่ต้องการ และตรงกับรูปแบบสำหรับนำเสนอสภามหาวิทยาลัย

In [2]:
cols = ['ตำแหน่งทางวิชาการ', 'ตำแหน่งจากการจบการศึกษาระดับปริญญาเอก', 'คำนำหน้านาม', 'ชื่อ', 'สกุล', 'คุณวุฒิ', 'มหาวิทยาลัย', 'คณะ', 'ความเชี่ยวชาญ', \
              'สถานที่ติดต่อ', 'ช่องทางการติดต่อ (เบอร์โทรศัพท์)', 'ช่องทางการติดต่อ (e-mail address)', 'สถานะ']
new_cols = ['ตำแหน่ง', 'ตำแหน่ง ป.เอก', 'คำนำหน้านาม', 'ชื่อ', 'นามสกุล', 'คุณวุฒิ', 'มหาวิทยาลัย', 'สาขาวิชา', 'ความเชี่ยวชาญ', \
              'สถานที่ติดต่อ', 'โทร', 'email', 'สถานะ']

### รวมข้อมูลจากทุก Sheet

ในไฟล์ Excel ข้อมูลในแต่ละ Sheet แบ่งตามสถาบันที่ผู้ทรงคุณวุฒิสังกัด

ในแต่ละ Sheet ทำดังนี้
* ลบแถวที่ไม่มีข้อมูล
* แยกข้อมูลที่มีปัญหาออกมา และบันทึกในไฟล์ที่กำหนด
* เพิ่มคอลัมน์ สถานะ คุณวุฒิ ความเชี่ยวชาญ และสถานที่ติดต่อ แต่ไม่มีข้อมูลเหล่านี้
* เพิ่มคอลัมน์ มหาวิทยาลัยตามชื่อ Sheet

สุดท้าย เปลี่ยนชื่อคอลัมน์ใหม่

In [3]:
dfs = None
for i, sheet in enumerate(xl.sheet_names):
    print(sheet)
    df = xl.parse(sheet)
    df = df.dropna(how='all')
    df = rmblankname(df, cols[3:5])

    df['สถานะ'] = ''
    df['มหาวิทยาลัย'] = sheet
    df["คุณวุฒิ" ] =""
    df['ความเชี่ยวชาญ'] = ''
    df['สถานที่ติดต่อ'] = ''
    df['คำนำหน้านาม'] = ''
    df = df[cols]

    dfs = pd.concat([dfs, df])
dfs.columns = new_cols

จุฬาลงกรณ์มหาวิทยาลัย
มหาวิทยาลัยมหิดล
มหาวิทยาลัยเกษตรศาสตร์
มหาวิทยาลัยศรีนครินทรวิโรฒ
มหาวิทยาลัยศิลปากร
มหาวิทยาลัยเชียงใหม่
มหาวิทยาลัยนิด้า
พระจอมเกล้าเจ้าคุณทหารลาดกระบัง
พระจอมเกล้าพระนครเหนือ


In [4]:
dfs['ตำแหน่ง'].unique()

array(['ผู้ช่วยศาสตราจารย์', 'รองศาสตราจารย์', nan, 'ศาสตราจารย์',
       'รองศาสตราจารย์ (พิเศษ)', 'ศาสตราจารย์ กิตติคุณ', 'ศาสตรจารย์',
       'รองศาสตร์ตราจารย์', 'ศาสตร์ตราจารย์', '\xa0ศาสตราจารย์',
       'ศาสตราจารย์เกียรติคุณ', 'ศาสตราจารย์พิเศษ', 'Associate Professor',
       '\xa0รองศาสตราจารย์'], dtype=object)

### การแก้ไขข้อมูล
* แก้ชื่อตำแหน่ง, ตำแหน่ง ป.เอก, ชื่อมหาวิทยาลัย ที่ปนระหว่างชื่อเต็ม และตัวย่อ หรือสะกดไม่เหมือนกัน
* แก้ สถานะ ให้มีแค่ เกษียณ, ลาออก, หรือปกติ
* แก้ชื่อ โดยการลบคำนำหน้าชื่อ ทั้งที่เป็นตัวย่อ และคำเต็ม เช่น นาย, นาง, นางสาว

In [5]:
dfs['ตำแหน่ง'] = dfs['ตำแหน่ง'].fillna('').apply(fixranks)
dfs['ตำแหน่ง ป.เอก'] = dfs['ตำแหน่ง ป.เอก'].fillna('').apply(fixphd)
dfs['มหาวิทยาลัย'] = dfs['มหาวิทยาลัย'].fillna('').apply(fixuni)
dfs['สถานะ'] = dfs['สถานะ'].fillna('').apply(fixstatus)

In [6]:
dfs['คำนำหน้านาม'] = dfs['ชื่อ'].fillna('').apply(get_title)

In [7]:
for row in dfs[['ชื่อ', 'คำนำหน้านาม']].iterrows():
    if row[1]['คำนำหน้านาม']:
        print(row[1]['คำนำหน้านาม'], '\t', row[1]['ชื่อ'])

มล. 	 มล.ปิยลดา
พ.ต.อ. 	 พ.ต.อ.หญิง เอื้อญาติ
นพ. 	 นพ. ไพโรจน์
นพ. 	 นพ. วรพจน์
พญ. 	 พญ. ทวิมา
นพ. 	 นพ. อรุชา
ทพ. 	 ทพ.สรสัณห์
ม.ร.ว. 	 ม.ร.ว.สุริยวุฒิ
ร.ต.อ. 	 ร.ต.อ. ดร.อนุชา
ภญ. 	 ภญ. นุศรา
ภญ. 	 ภญ. อวยพร
ภญ. 	  ภญ.นลินี
ภก. 	 ภก. มนัส
ภก. 	 ภก. วิชัย
ภญ. 	 ภญ. ระพีพรรณ
ภญ. 	 ภญ. จันคนา
ภญ. 	 ภญ. ชุติมา
ภก. 	 ภก. ธีรศักดิ์
ภญ. 	 ภญ. ปนัดดา
ภญ. 	 ภญ. ลาวัลย์
ภญ. 	 ภญ. อรอุมา
ภก. 	 ภก. ฉัตชัย
ภญ. 	 ภญ. ศรีสมบัติ
ภก. 	 ภก. อุทัย
ภญ. 	 ภญ. เพ็ญพรรณ
ภญ. 	 ภญ. จุไรรัตน์
ภก. 	 ภก. ธนะเศรษฐ์
ภก. 	 ภก. ธวัชชัย
ภก. 	 ภก. ประเสริฐ
ภญ. 	 ภญ. มานี
ภก. 	 ภก. สนทยา
ภก. 	 ภก. สมลักษณ์
ภญ. 	 ภญ. สุชาดา
ภญ. 	 ภญ. สุวรรณี
ภญ. 	 ภญ. เกษร
ภก. 	 ภก. วีรยุทธ์ 
นพ. 	 นพ.พงษ์รักษ์
นพ. 	 นพ.อำนาจ
นพ. 	 นพ. อำนาจ 
ภก. 	 ภก. วิรัตน์
ภญ. 	 ภญ. ศิริวิภา
ภญ. 	 ภญ. หทัยกาญจน์
ภญ. 	 ภญ. ชบาไพร
ภก. 	 ภก. โชคชัย
ภญ. 	 ภญ. เพ็ญศักดิ์
ภญ. 	 ภญ. อัญชลี 
ทพญ. 	 ทพญ. สั่งสม
นพ. 	 นพ. พงษ์รักษ์
นพ. 	 นพ. ขวัญชัย
นพ. 	 นพ.เกรียงไกร
นพ. 	 นพ.กิตติพันธุ์
นพ. 	 นพ. สุวัฒน์
ภญ. 	 ภญ.มาลิน


In [8]:
dfs['ชื่อ'] = dfs['ชื่อ'].fillna('').apply(fix_name)
dfs['ชื่อ'] = dfs['ชื่อ'].fillna('').apply(fix_title)

In [9]:
dfs['ตำแหน่ง ป.เอก'].unique()

array(['ดร.', ''], dtype=object)

In [10]:
dfs['ตำแหน่ง'].unique()

array(['ผู้ช่วยศาสตราจารย์', 'รองศาสตราจารย์', '', 'ศาสตราจารย์',
       'รองศาสตราจารย์ (พิเศษ)', 'ศาสตราจารย์กิตติคุณ',
       'ศาสตราจารย์เกียรติคุณ', 'ศาสตราจารย์ (พิเศษ)'], dtype=object)

In [11]:
dfs['คำนำหน้านาม'].unique()

array(['', 'มล.', 'พ.ต.อ.', 'นพ.', 'พญ.', 'ทพ.', 'ม.ร.ว.', 'ร.ต.อ.',
       'ภญ.', 'ภก.', 'ทพญ.'], dtype=object)

In [12]:
dfs

,ตำแหน่ง,ตำแหน่ง ป.เอก,คำนำหน้านาม,ชื่อ,นามสกุล,คุณวุฒิ,มหาวิทยาลัย,สาขาวิชา,ความเชี่ยวชาญ,สถานที่ติดต่อ,โทร,email,สถานะ
0,ผู้ช่วยศาสตราจารย์,ดร.,,สุรภี,รุโจปการ,,จุฬาลงกรณ์มหาวิทยาลัย,ครุศาสตร์,,,02-218-2565 กด 0 (เบอร์โทรศัพท์คณะ),edcu57@gmail.com (e-mail address คณะ),
1,ผู้ช่วยศาสตราจารย์,ดร.,,อภิชาติ,พลประเสริฐ,,จุฬาลงกรณ์มหาวิทยาลัย,ครุศาสตร์,,,02-218-2565 กด 0 (เบอร์โทรศัพท์คณะ),edcu57@gmail.com (e-mail address คณะ),
2,รองศาสตราจารย์,ดร.,,จรูญศรี,มาดิลกโกวิท,,จุฬาลงกรณ์มหาวิทยาลัย,ครุศาสตร์,,,02-218-2565 กด 0 (เบอร์โทรศัพท์คณะ),edcu57@gmail.com (e-mail address คณะ),
3,รองศาสตราจารย์,ดร.,,วรรณี,แกมเกตุ,,จุฬาลงกรณ์มหาวิทยาลัย,ครุศาสตร์,,,02-218-2565 กด 0 (เบอร์โทรศัพท์คณะ),edcu57@gmail.com (e-mail address คณะ),
4,รองศาสตราจารย์,ดร.,,ใจทิพย์,ณ สงขลา,,จุฬาลงกรณ์มหาวิทยาลัย,ครุศาสตร์,,,02-218-2565 กด 0 (เบอร์โทรศัพท์คณะ),edcu57@gmail.com (e-mail address คณะ),
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,รองศาสตราจารย์,,,สมจิตร,วิริยานนท์,,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าพระนครเหนือ,คณะศิลปศาสตร์ประยุกต์,,,"02-5872096 , 02-5552000 (เบอร์โทรศัพท์คณะ)",somjitw@kmutnb.ac.th,
136,รองศาสตราจารย์,ดร.,,สักรินทร์,อยู่ผ่อง,,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าพระนครเหนือ,คณะศิลปศาสตร์ประยุกต์,,,"02-5872096 , 02-5552000 (เบอร์โทรศัพท์คณะ)",skr@kmutnb.ac.th,
137,รองศาสตราจารย์,ดร.,,เสาวนิตย์,กาญจนรัตน์,,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าพระนครเหนือ,คณะสถาปัตยกรรมและการออกแบบ,,,"02-555-2000 สำนักงานคณบดี ต่อ 6810, วิชาการ ต่...",saowanit.k@archd.kmutnb.ac.th,
138,รองศาสตราจารย์,ดร.,,ชาญชัย,ทองประสิทธิ์,,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าพระนครเหนือ,คณะสถาปัตยกรรมและการออกแบบ,,,"02-555-2000 สำนักงานคณบดี ต่อ 6810, วิชาการ ต่...",chanchai.t@archd.kmutnb.ac.th,


บันทึกลงไฟล์ CSV

In [13]:
dfs.to_csv('TUlist.csv', encoding='utf-8-sig')